In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [61]:
%ls
%cd 'UCI HAR Dataset/'
%ls

 __MACOSX/  'UCI HAR Dataset'/
/media/ali/Ali/Projects/Machine_learning/RNN/UCI HAR Dataset/UCI HAR Dataset
activity_labels.txt*  features.txt*  test/
features_info.txt*    README.txt*    train/


In [34]:
def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# Loading data set

In [35]:
def load_group(filenames, prefix=''):
    loaded = list()
    for name in filenames:
        data = load_file(prefix + name)
        loaded.append(data)
        
    loaded = np.dstack(loaded)
    return loaded


In [36]:
# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

In [62]:
# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    from tensorflow.keras.utils import to_categorical
    # load all train
    trainX, trainy = load_dataset_group('train')
    print(trainX.shape, trainy.shape)
    # load all test
    testX, testy = load_dataset_group('test')
    print(testX.shape, testy.shape)
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print(trainX.shape, trainy.shape, testX.shape, testy.shape)
    return trainX, trainy, testX, testy

In [63]:
trainX, trainy, testX, testy = load_dataset()

(7352, 128, 9) (7352, 1)
(2947, 128, 9) (2947, 1)
(7352, 128, 9) (7352, 6) (2947, 128, 9) (2947, 6)


# Implementing the RNN

In [64]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [68]:
n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
model = Sequential([
    LSTM(100, input_shape=(n_timesteps, n_features)),
    Dropout(0.5),
    Dense(100, activation='relu'),
    Dense(n_outputs, activation='softmax' )
])

In [70]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [81]:
evaluate_model(trainX, trainy, testX, testy)

Epoch 1/15
7352/7352 [==============================] - 11s 2ms/step - loss: 1.1536 - acc: 0.5220
Epoch 2/15
7352/7352 [==============================] - 10s 1ms/step - loss: 0.7493 - acc: 0.6695
Epoch 3/15
7352/7352 [==============================] - 10s 1ms/step - loss: 0.6424 - acc: 0.7235
Epoch 4/15
7352/7352 [==============================] - 10s 1ms/step - loss: 0.6819 - acc: 0.6919
Epoch 5/15
7352/7352 [==============================] - 10s 1ms/step - loss: 0.5306 - acc: 0.7890
Epoch 6/15
7352/7352 [==============================] - 10s 1ms/step - loss: 0.4252 - acc: 0.8523
Epoch 7/15
7352/7352 [==============================] - 10s 1ms/step - loss: 0.4281 - acc: 0.8598
Epoch 8/15
7352/7352 [==============================] - 10s 1ms/step - loss: 0.3084 - acc: 0.8868
Epoch 9/15
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1949 - acc: 0.9344
Epoch 10/15
7352/7352 [==============================] - 10s 1ms/step - loss: 0.1782 - acc: 0.9359
Epoch 11/15
7352/73

0.9083814048184594

In [98]:
run_experiment(trainX, trainy, testX, testy, repeats=10)

2947/2947 [==============================] - 1s 450us/step
>#1: 90.024
2947/2947 [==============================] - 1s 470us/step
>#2: 89.854
2947/2947 [==============================] - 2s 782us/step
>#3: 90.092
2947/2947 [==============================] - 1s 500us/step
>#4: 90.974
2947/2947 [==============================] - 2s 520us/step
>#5: 83.407
2947/2947 [==============================] - 2s 538us/step
>#6: 90.668
2947/2947 [==============================] - 2s 556us/step
>#7: 91.788
2947/2947 [==============================] - 2s 569us/step
>#8: 91.585
2947/2947 [==============================] - 2s 620us/step
>#9: 89.990
2947/2947 [==============================] - 2s 605us/step
>#10: 88.802
[90.02375296912113, 89.85408890397014, 90.09161859518154, 90.97387173396675, 83.40685442823211, 90.66847641669494, 91.78825924669155, 91.58466236851035, 89.98982015609094, 88.80217170003394]


NameError: name 'scores55' is not defined

In [95]:
def evaluate_model(trainX, trainy, testX, testy):
    epochs, batch_size = 15, 64
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = Sequential([
        LSTM(100, input_shape=(n_timesteps, n_features)),
        Dropout(0.5),
        Dense(100, activation='relu'),
        Dense(n_outputs, activation='softmax'),
 ])
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=0)
    
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size)
    
    return accuracy


In [99]:
def summarize_results(scores):
    print(scores)
    m, s = np.mean(scores), np.std(scores)
    print('Accuracy: %.3f%% (+/- %.3f)' % (m,s))

In [97]:
def run_experiment(trainX, trainy, testX, testy, repeats=10):
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
        
    summarize_results(scores)